In [45]:
import jieba
import logging
import os
import sys
import multiprocessing

from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences

import warnings
import gensim

In [46]:
def seg_words():
    origin_file = "data/cleaned_tex.tex"  # 原始文件
    stop_words_file = "data/hit_stopwords.txt"  # 停用词路径
    user_dict_file = "data/jieba_dict.txt"  # 用户自定义词典路径
    stop_words = list()
    # 加载停用词
    with open(stop_words_file, 'r', encoding="utf8") as stop_words_file_object:
        contents = stop_words_file_object.readlines()
        for line in contents:
            line = line.strip()
            stop_words.append(line)
    # 加载用户字典(停用)
    jieba.load_userdict(user_dict_file)
    target_file = open("data/cut_word.txt", 'w', encoding="utf-8")
    with open(origin_file, 'r', encoding="utf-8") as origin_file_object:
        contents = origin_file_object.readlines()
        for line in contents:
            line = line.strip()
            out_str = ''
            word_list = jieba.cut(line, cut_all=False)
            for word in word_list:
                if word not in stop_words:
                    if word != "\t":
                        out_str += word
                        out_str += ' '
            target_file.write(out_str.rstrip() + "\n")
    target_file.close()

    print("end")
seg_words()

end


In [47]:
def train_model():
    program = os.path.basename(sys.argv[0])
    logger = logging.getLogger(program)
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    logger.info("running %s" % ' '.join(sys.argv))

    input_dir = 'data/cut_word.txt'
    outp1 = 'model/vector.model'
    outp2 = 'model/vector.vector'

    model = Word2Vec(PathLineSentences(input_dir),
                     size=100, min_count=2,
                     #window= 10,
                     workers=multiprocessing.cpu_count(), iter=10,sg=1)
    model.save(outp1)	# 存储二进制模型文件
    model.wv.save_word2vec_format(outp2, binary=False)	# 存储类似于数组的模型文件
train_model()

2023-06-08 21:56:30,430: INFO: running C:\Anaconda\envs\torch\lib\site-packages\ipykernel_launcher.py -f C:\Users\12929\AppData\Roaming\jupyter\runtime\kernel-094d46b2-f88d-4eda-8b0b-dfd203a5c50f.json
2023-06-08 21:56:30,431: INFO: files read into PathLineSentences:data/cut_word.txt
2023-06-08 21:56:30,432: INFO: collecting all words and their counts
2023-06-08 21:56:30,433: INFO: reading file data/cut_word.txt
2023-06-08 21:56:30,439: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-08 21:56:30,445: INFO: collected 1617 word types from a corpus of 12099 raw words and 495 sentences
2023-06-08 21:56:30,445: INFO: Loading a fresh vocabulary
2023-06-08 21:56:30,448: INFO: effective_min_count=2 retains 899 unique words (55% of original 1617, drops 718)
2023-06-08 21:56:30,449: INFO: effective_min_count=2 leaves 11381 word corpus (94% of original 12099, drops 718)
2023-06-08 21:56:30,452: INFO: deleting the raw counts dictionary of 1617 items
2023-06-08 21:56:

In [48]:
warnings.filterwarnings(action='ignore', category=UserWarning,module='gensim')
model = gensim.models.Word2Vec.load("model/vector.model")

word = '集合'
result = model.wv.most_similar(word)

print("跟 "+word+" 最相近的词：")
for i in result:
    print(i)

2023-06-08 21:57:26,477: INFO: loading Word2Vec object from model/vector.model
2023-06-08 21:57:26,493: INFO: loading wv recursively from model/vector.model.wv.* with mmap=None
2023-06-08 21:57:26,493: INFO: setting ignored attribute vectors_norm to None
2023-06-08 21:57:26,494: INFO: loading vocabulary recursively from model/vector.model.vocabulary.* with mmap=None
2023-06-08 21:57:26,494: INFO: loading trainables recursively from model/vector.model.trainables.* with mmap=None
2023-06-08 21:57:26,496: INFO: setting ignored attribute cum_table to None
2023-06-08 21:57:26,496: INFO: loaded model/vector.model
2023-06-08 21:57:26,499: INFO: precomputing L2-norms of word weight vectors


跟 集合 最相近的词：
('给定', 0.9876338243484497)
('二元运算', 0.9689164757728577)
('关系', 0.968582272529602)
('是否', 0.9623192548751831)
('一定', 0.9616271257400513)
('唯一', 0.9577083587646484)
('偏序集', 0.9568491578102112)
('满足', 0.9537926912307739)
('关系图', 0.9524098634719849)
('等值', 0.9521253705024719)


In [49]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_product = norm(vector1) * norm(vector2)
    similarity = dot_product / norm_product
    return similarity

vec1 = model['运算'] - model['二元运算']
vec2 = model['度数'] - model['出度']

cosine_similarity(vec1,vec2)

C:\Users\12929\AppData\Local\Temp\ipykernel_13036\889666357.py:10: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\12929\AppData\Local\Temp\ipykernel_13036\889666357.py:11: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



0.8555767

In [50]:
result = model.wv.similar_by_vector(model['出度'] -  model['度数'] + model['无向图'],topn=5)
result

C:\Users\12929\AppData\Local\Temp\ipykernel_13036\1940268664.py:1: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



[('关联', 0.9792715907096863),
 ('无向图', 0.9769973158836365),
 ('次数', 0.9750767350196838),
 ('顶点集', 0.9744791984558105),
 ('边关联', 0.9742772579193115)]

In [51]:
model.wv.vocab

{'{': <gensim.models.keyedvectors.Vocab at 0x23acc4650d0>,
 '1': <gensim.models.keyedvectors.Vocab at 0x23acc465040>,
 '章': <gensim.models.keyedvectors.Vocab at 0x23abef9bfd0>,
 '命题逻辑': <gensim.models.keyedvectors.Vocab at 0x23acde06940>,
 '内容提要': <gensim.models.keyedvectors.Vocab at 0x23acde06610>,
 '命题': <gensim.models.keyedvectors.Vocab at 0x23acde06b20>,
 '联结词': <gensim.models.keyedvectors.Vocab at 0x23acde06730>,
 '真值': <gensim.models.keyedvectors.Vocab at 0x23acde06760>,
 '不是': <gensim.models.keyedvectors.Vocab at 0x23acde06be0>,
 '真': <gensim.models.keyedvectors.Vocab at 0x23acde06640>,
 '假': <gensim.models.keyedvectors.Vocab at 0x23acde066d0>,
 '陈述句': <gensim.models.keyedvectors.Vocab at 0x23acde06d90>,
 '称为': <gensim.models.keyedvectors.Vocab at 0x23acde06b80>,
 '判断': <gensim.models.keyedvectors.Vocab at 0x23acde06af0>,
 '两个': <gensim.models.keyedvectors.Vocab at 0x23acde06a30>,
 '值': <gensim.models.keyedvectors.Vocab at 0x23acde06a90>,
 '真的': <gensim.models.keyedvectors.Vocab

In [52]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)
plt.show()
